# Parameter Identification for Data from Different Distributions - Standard Method

This notebook introduces a parameter identification method for identifying parameters of data from different distributions. The approach is based on the empirical cumulative distribution function (eCDF). The notebook demonstrates how the method works with different distributions. First, data is created using known parameters. Subsequently, new data is generated using various test parameters to identify the initial parameter. The implementation of the method using Python and `ecdf-estimator` package is explained step-by-step within this notebook.

## Setup of the Process

###Environment Setup

First, `ecdf-estimator` is installed. It contains a Python implementation of the eCDF based approach which will be used. Basically, the main functionalities in the method are done with functions included in this module. The installation is needed because the module is not pre-installed in Jupyter Notebook or other Python environments used.

In [ ]:
pip install ecdf-estimator

After the installation, the estimator is imported and abbreviated as `ecdf`.


In [ ]:
import ecdf_estimator as ecdf

Next, the other two necessary modules, `numpy` and `matplotlib.pyplot`, are imported. The modules are used to do array operations efficiently and to visualize the data and results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Creation of Training Data

Next,  training data must be created. First, the variables which are used to create the data are defined:

* `initParam`: The initial parameter of training data, which is tried to identify. <br>
* `size`: The number of created data points.<br>


In [ ]:
initParam = 5
size = 5000

The variable `dist` defines the distribution of the training data:

* `exp`: exponential distribution, identified parameter is the scale parameter
* `poisson`: poisson distribution, identified parameter is the expectation of interval
* `t`: standard student's t distribution, identified parameter is the degree of freedoms
* `gamma`: gamma distribution, identified parameter is the shape of the distribution (and scale = 1)
* `chi2`: chi-square distribution, identified parameter is the degree of freedoms
* `rayleigh`: Rayleigh distribution, identified parameter is the scale parameter.


In [ ]:
dist = "exp"
#dist = "poisson"
#dist = "t"
#dist = "gamma"
#dist = "chi2"
#dist = "rayleigh"

After the parameters are defined, the training data can be created using `numpy`'s `random` functions, which take two input parameters: the initial parameter to be identified and the size of data.

In [ ]:
if dist == "exp":
  data = np.random.exponential(initParam, size)
elif dist == "poisson":
  data = np.random.poisson(initParam, size)
elif dist == "t":
  data = np.random.standard_t(initParam, size)
elif dist == "gamma":
  data = np.random.standard_gamma(initParam, size)
elif dist == "chi2":
  data = np.random.chisquare(initParam, size)
elif dist == "rayleigh":
  data = np.random.rayleigh(initParam, size)
else:
  print("Invalid data distribution.")

## Parameter Identification Process

After the creation of training data, the actual parameter identification begins.

### Definition of Variables

The variables which will be used in the process are defined.

* `subset_sizes`: The list of subset sizes of training data. By default, the training data will be split into 20 subsets, and each subset's size is 250. Note that the sum of the sizes of subsets should equal the size of training data!

In [ ]:
subset_sizes = [250]*20

* `n_bins`: The maximum number of bins which will be used for the eCDF vectors. The exact number will be defined by the `ecdf`'s own functions.

In [ ]:
n_bins = 7

* `min_param`: The start parameter to be estimated.<br>
* `max_param`: The end parameter to be estimated.<br>

Now, only integers greater than zero are tested because they can be parameters for each distribution.

In [ ]:
min_param = 2
max_param = 8

Next, a function named `distance`, which has a major part in the creation of eCDF vectors in the whole identification process, must be defined. The function computes the absolute value of the subtraction of the data points, which is the so called Euclidian distance between the points.

In [ ]:
def distance(data_a, data_b):
  return np.abs(data_a-data_b)

###Generation of Objective Functions and eCDF vectors

Next, the eCDF vectors have to be created and plotted. This is done by initializing a new instance of `ecdf`'s `standard` class object function. Because the vectors are wanted to plot using a large number of bins and with smaller amount of bins, they must be created twice.

The `ecdf-estimator`'s `estimate_radii_values` is called to determine appropriate region values for the bin values. The values are selected based on the computed distances between data points of the first two subsets of the training data. The returned region values are stored to `min_val` and `max_val` and the distances between data points to the matrix `distance_data`.

In [ ]:
min_val, max_val, distance_data = ecdf.estimate_radii_values(data, subset_sizes, distance)

Then, the following interval is split to 50 bins using `numpy`'s `linspace` function. It takes the region points of the interval and the number of equally spaced values between the points as input arguments and returns the values as a list, now named as `bins`.

In [ ]:
bins = np.linspace(min_param, max_param, 50)

The objective function is assembled by initializing a new instance of `estimator`'s `standard` class object. `data`, `bins`, `distance` function and `subset_sizes` are given as input arguments. The distances between all subsets of the training data are calculated. Thus, for each possible subset pair, each possible distance between two data points of the pairs is computed with the `distance` function. Then, the eCDF vector of each subset pair is created by cumulatively calculating how many of these distances belong to a certain of the 50 bins. Also, the mean and covariance of all vectors are computed. All these values are stored to `aux_func` to specify the first objective function with all 50 bins. This function is only used for plotting and for that reason named as an auxiliary function.

In [ ]:
aux_func = ecdf.standard(data, bins, distance, subset_sizes)

Then, the other function with smaller amount of bins must be defined. Otherwise there could be unwanted correlation between neighbouring bin values. The `estimator`'s `choose_bins` function is called to select the reasonable bin values from larger choice. Note that the previously selected 50 bin values, `bins`, is given as an input argument and overwritten with a new list of maximum of `n_bins` values. Third input argument is the matrix `distance_data`, which was computed earlier.

In [ ]:
bins = ecdf.choose_bins(distance_data, bins, n_bins)

Now, the second objective function `func` can be defined. The only difference compared to the creation of `aux_func` is that this time only the small amount of bins is used to create the eCDF vectors and compute the statistics for them. This function is used for plotting and evaluation of different parameters.

In [ ]:
func = ecdf.standard(data, bins, distance, subset_sizes)

After the two objective functions have been created, they can be plotted. First, the figure `fig` and its axes `ax` have to be defined using `matplotlib.pyplot`'s `subplots` function. The dimensions of the figure are defined with an input argument `figsize=(12, 4)`. Then, the plots are made using the `estimator`'s functions, which take three input arguments: the objective function containing the eCDF vectors and their statistics, axes of the figure to plot on, and the plotting style (colour and shape, which is now small circles). The vectors with 50 bins are plotted as purple, with `aux_func` as an input argument. The ones with selected bins only are plotted as light blue and their mean values as black. Then, title and labels are defined. The `plt.show()` command ensures that the figure is displayed properly without unwanted prints.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ecdf.plot_ecdf_vectors(aux_func, ax, 'm.')
ecdf.plot_ecdf_vectors(func, ax, 'c.')
ecdf.plot_mean_vector(func, ax, 'k.')
plt.title("Distribution of the eCDF-vectors")
plt.xlabel("Bin Values")
plt.ylabel("Cumulative Probability")
plt.show()

###$\chi^2$ test: Checking normality of eCDF vectors

The eCDF vectors should be Gaussian with big enough sample size of training data. This is ensured using the $\chi^2$ test which is done by calling the `estimator`'s `plot_chi2_test` function. The function computes the negative log-likelihood values for each eCDF vector to examine how well each vector fits the objective function `func`'s model. The vectors being multinormally distributed, log-likelihood values can be computed with the help of mean and covariance values of the objective function. The log-likelihood values are then normalized and a histogram of them is created and plotted. After that, the probability density function of the chi-square distribution with an appropriate degree of freedom (which is the number of the eCDF vectors) is plotted on top of the histogram. If the histogram fits the density function well, the normality of eCDF vectors is confirmed and the parameter estimation process should be valid and reliable.

Finally, the title and labels are defined and the correct display of the figure with plots is ensured.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ecdf.plot_chi2_test(func, ax)
plt.title("Gaussianity Test by Chi-squared -criterion")
plt.xlabel("Normalized Log-likelihood")
plt.ylabel("Probability Density")
plt.show()

###Evaluation of Parameters

After ensuring the reliability of the identification process, the evaluation of different parameters begins. First, a list `params`, which includes all the parameters which will be evaluated, is created. Command `range(min_param, max_param + 1)` defines the correct range of integer values based on the region points, and `list` creates a list of the values. Then, the number of parameters is stored to `n_params` by computing the length of the list.



In [ ]:
params = list(range(min_param, max_param + 1))
n_params = len(params)

Next, a list of lists, or a matrix, `values`, is initialized. When referring to a list of lists as a matrix, one list can be thought as a row of a matrix. The for loop creates a new empty list `n_params` times, one for each parameter. Note that the `range` command starts from 0 if only one input argument is given. The matrix will store objective function values, which are negative log-likelihood values, for datasets created for each parameter. One row of the matrix will contain the values for one parameter. A single value of a matrix can be selected with command `matrix[i][j]`, `i` being the row index and `j` the column index of a matrix. As mentioned, indexing starts from 0.

List `means_log` is initialized as zeros with a length of `n_params`, and will store mean log-likelihood values for each parameter.

In [ ]:
values = [[] for i in range(n_params)]
means_log = [0.] * n_params

A variable `n_newsets` defines how many new datasets are created for each parameter and evaluated with the objective function. By default, 5 subsets are evaluated for each parameter.

In [ ]:
n_newsets = 5

####Negative Log-likelihood Analysis

The creation and evaluation of new datasets starts and the negative log-likelihood values will be computed. `n_subsets` new datasets are wanted to create for each parameter. Thus, the outer loop iterates `n_params` times and the inner loop `n_subset` times. This way it is easy to refer to a current parameter using `params[i]` or to a current subset size using `subset_sizes[j]`. In each iteration loop, a new dataset `newdata` with the current parameter `params[i]` is created using numpy's `random.normal`-function, just like with the creation of training data. The size of a new dataset is the same as the similarly indexed dataset's size in training data, `subset_sizes[j]`.

Then, the `estimator`'s `evaluate` function is called to evaluate the new dataset with the objective function `func`. First, one subset of the training data is selected and the distances between each point of the training data subset and the new dataset are computed. Again, an eCDF function of the distances is created, similarly to what was done before with training data subsets. Then, the negative log-likelihood value of the new eCDF vector is calculated to examine how it fits the objective function created with training data. This value is the one which the function returns, and it is stored into matrix `values` by appending it to the end of the current parameter's row `i` of the matrix with `values[i].append()`. When the inner loop is exited, the negative log-likelihood values for all datasets with one parameter are computed. Thus, the mean value of that row is computed with `numpy`'s `mean` function and stored in list `means_log`.

In [ ]:
for i in range(n_params):
  for j in range(n_newsets):
    if dist == "exp":
      newdata = np.random.exponential(params[i], subset_sizes[j])
    elif dist == "poisson":
      newdata = np.random.poisson(params[i], subset_sizes[j])
    elif dist == "t":
      newdata = np.random.standard_t(params[i], subset_sizes[j])
    elif dist == "gamma":
      newdata = np.random.standard_gamma(params[i], subset_sizes[j])
    elif dist == "chi2":
      newdata = np.random.chisquare(params[i], subset_sizes[j])
    elif dist == "rayleigh":
      newdata = np.random.rayleigh(params[i], subset_sizes[j])
    else:
      print("Invalid data distribution.")

    values[i].append(ecdf.evaluate(func, newdata))

  means_log[i] = np.mean(values[i])

The evaluated log-likelihood values for all datasets and their means are plotted. Again, the figure and axes are defined. This time, the plots are made using `matplotlib.pyplot`'s ready `plot` function, which takes the x and y -coordinates as input arguments with a plotting style. Every dataset's value is plotted with red and the means for each parameter with black. The smaller the mean value is to a certain parameter, the better the parameter fits the training data.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(params, values, 'ro')
ax.plot(params, means_log, 'bo')
plt.title("Evaluation of the Negative Log-likelihood Values")
plt.xlabel("Tested Parameter")
plt.ylabel("Log-likelihood Value")
plt.show()

####Normalization of Log-likelihood Values

The final step of the process is the normalization of the log-likelihood values. First, each value is multiplied with -0.5 and then exponentiated. This operation is now suitable because it turns the smallest negative log-likelihood values, which means the best fits to training data, to the biggest values. The operation is done with `numpy`'s `exp` function. The previous nested loops are now replaced with list comprehension, which tend to be faster in Python. The inner loop for each subset has to be before the outer loop for each parameter, and the square brackets have to be used between the loops, so that the operations are done correctly to each element of the matrix.

In [ ]:
values = [[np.exp(-0.5*values[i][j]) for j in range(n_newsets)] for i in range(n_params)]

Then, the sum of each column, which all include one log-likelihood value for each parameter, is computed and stored in list `sums`. This is done with `numpy`'s `sum`-function. When using `axis=0` as a second input argument, it computes the sum of columns, and returns it as a list which is wanted. Without the second input argument it would compute the sum of all elements, and with the second input argument `axis=1` it would compute the sum of rows.

In [ ]:
sums = np.sum(values, axis=0)

After that, each value in `values` is divided by the sum of its' column to get the likelihood values for each parameter in the column to fit the training data. Basically, one dataset is selected for each parameter, and the likelihood value for each dataset is computed. This is done `n_newsets` times, so that all datasets are used. The calculation is done using list comprehension as before.

In [ ]:
values = [[values[i][j] / sums[j] for j in range(n_newsets)] for i in range(n_params)]

The means of each row, which are the means of the datasets' likelihood values for each parameter, are computed. The means are computed with `numpy`'s `mean` function, where every row of the matrix is put as an input argument, using list comprehension. The mean values are stored to list `means_nor`. Note the difference compared to previous list comprehensions when now a list is used instead of a matrix.

In [ ]:
means_nor = [np.mean(values[i]) for i in range(n_params)]

Finally, the normalized likelihood values (red) and the means of them (blue) for each parameter are plotted as before, and the goodness of each parameter can be estimated.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(params, values, 'ro')
ax.plot(params, means_nor, 'bo')

plt.title("Normalized Likelihood Values and Average Values Over All Evaluations")
plt.xlabel("Tested Parameter")
plt.ylabel("Likelihood")
plt.show()